In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, 
import itertools
#print('We will use RAPIDS version',cudf.__version__)
import warnings
warnings.filterwarnings('ignore')

# test load

In [14]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/candidate/popular/'

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(INPUT_DIR+'cris_baseline/otto-validation/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        #chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

#test_df
test_df = load_test()
print('Test data has shape',test_df.shape)
test_df

Test data has shape (7683577, 4)


,session,aid,ts,type
0,11188591,1485015,1661165822000,0
1,11188591,1562739,1661165866000,0
2,11188591,1485015,1661165882000,0
3,11188591,1441634,1661165891000,0
4,11188591,1485015,1661165901000,0
...,...,...,...,...
7683572,12629597,744187,1661646518000,0
7683573,12629597,714524,1661646521000,0
7683574,12629597,1844379,1661646612000,0
7683575,12629598,894387,1661646273000,0


#debug

In [5]:
#test_df = test_df.sample(100000).reset_index(drop=True)
#test_df

#test dayごとの人気商品

In [6]:
%%time
import collections
from tqdm import tqdm

#https://www.kaggle.com/code/crained/otto-breaking-out-the-dates
test_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')
test_df['day'] = test_df['date'].dt.strftime('%d')

test_days = [21, 22, 23, 24, 25, 26, 27, 28]
top_aids = 20
test_days_click_cart_order_popular_aids = {}

for day in tqdm(test_days):
  aids = []
  for aid in test_df.groupby('day')['aid'].value_counts().index.values:
    if len(aids) < top_aids:
      if day == int(aid[0]):
        aids.append(aid[1])
    continue
  test_days_click_cart_order_popular_aids[day] = aids

100%|██████████| 8/8 [00:45<00:00,  5.66s/it]

CPU times: user 1min 20s, sys: 1.8 s, total: 1min 22s
Wall time: 1min 21s


In [7]:
test_days_click_cart_order_popular_aids

{21: [108125,
  1603001,
  29735,
  166037,
  558573,
  1505419,
  1581568,
  1768724,
  331708,
  1022566,
  1733943,
  1685214,
  337471,
  632365,
  1539309,
  137514,
  225619,
  544144,
  831165,
  984459],
 22: [1460571,
  166037,
  1733943,
  331708,
  108125,
  579690,
  1596897,
  836852,
  29735,
  554660,
  332654,
  195381,
  1022566,
  1604220,
  634452,
  184976,
  1552037,
  1645990,
  620545,
  530377],
 23: [485256,
  33343,
  152547,
  1551213,
  1765072,
  1736857,
  660655,
  803544,
  1562705,
  321547,
  1460571,
  184976,
  1236775,
  331708,
  579690,
  836852,
  1043508,
  108125,
  1604220,
  1733943],
 24: [613493,
  1406660,
  1551213,
  1460571,
  1236775,
  108125,
  1531805,
  184976,
  959208,
  1294924,
  19665,
  836852,
  738987,
  1347107,
  231487,
  29735,
  1022566,
  166037,
  554660,
  331708],
 25: [876493,
  1460571,
  108125,
  1236775,
  1551213,
  1531805,
  184976,
  836852,
  1116095,
  554660,
  331708,
  634452,
  231487,
  959208,
  16

#test sessionごとにdayごとの人気商品を予測

In [8]:
def test_popula_aids(df):
  day=int(df.day.max())
  return ' '.join(map(str, test_days_click_cart_order_popular_aids[day]))

In [9]:
%%time
pred_df = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(lambda x:test_popula_aids(x))
pred_df = pd.DataFrame(pred_df, columns=["labels"]).reset_index()
pred_df

CPU times: user 6min 44s, sys: 4.43 s, total: 6min 48s
Wall time: 6min 44s


,session,labels
0,11098528,108125 1603001 29735 166037 558573 1505419 158...
1,11098529,108125 1603001 29735 166037 558573 1505419 158...
2,11098530,108125 1603001 29735 166037 558573 1505419 158...
3,11098531,108125 1603001 29735 166037 558573 1505419 158...
4,11098532,108125 1603001 29735 166037 558573 1505419 158...
...,...,...
1801246,12899774,1460571 108125 184976 1844379 1006198 1531805 ...
1801247,12899775,1460571 108125 184976 1844379 1006198 1531805 ...
1801248,12899776,1460571 108125 184976 1844379 1006198 1531805 ...
1801249,12899777,1460571 108125 184976 1844379 1006198 1531805 ...


#test label

In [10]:
#test_labels
test_labels = pd.read_parquet('/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/otto-validation/test_labels.parquet')
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


#recall

In [11]:
%%time

#clicks,cart,orderのracall計算
def recall_k_evaluate(test_labels, pred_df, session_types, top_k):
    sub = pred_df.copy()
    #print(sub)
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')][:top_k])
    test_labels = test_labels.loc[test_labels['type']==session_types]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1) 
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,top_k)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print(f'{session_types} recall =',recall)

#clicksの場合
#test label(CV変わるときのため), pred_df_clicks, type, top_kは変わるので引数にする。
recall_k_evaluate(test_labels, pred_df, 'clicks', 20)

clicks recall = 0.021153677456545984
CPU times: user 56 s, sys: 1.42 s, total: 57.4 s
Wall time: 57.2 s


# save

In [15]:
with open(OUTPUT_DIR + 'test_days_click_cart_order_popular_aids.pkl', 'wb') as f:
  pickle.dump(test_days_click_cart_order_popular_aids, f)